In [2]:
from bids import BIDSLayout

In [4]:
data = BIDSLayout('../data/1604003015/bids_dataset/')

In [8]:
data.to_df()

entity,path,datatype,extension,fmap,session,subject,suffix
0,/storage/ttapera/RBC/data/1604003015/bids_data...,NaN,json,NaN,NaN,NaN,description
1,/storage/ttapera/RBC/data/1604003015/bids_data...,anat,json,NaN,PNC1,81231,T1w
2,/storage/ttapera/RBC/data/1604003015/bids_data...,anat,nii.gz,NaN,PNC1,81231,T1w
3,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,json,NaN,PNC1,81231,magnitude1
4,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,nii.gz,magnitude1,PNC1,81231,magnitude1
5,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,json,NaN,PNC1,81231,magnitude2
6,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,nii.gz,magnitude2,PNC1,81231,magnitude2
7,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,json,NaN,PNC1,81231,phase1
8,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,nii.gz,phase1,PNC1,81231,phase1
9,/storage/ttapera/RBC/data/1604003015/bids_data...,fmap,json,NaN,PNC1,81231,phase2


In [10]:
data.validate

True

In [13]:
t1 = data.get(suffix = 'T1w', extension='nii.gz')

In [27]:
t1[0].path

'/storage/ttapera/RBC/data/1604003015/bids_dataset/sub-81231/ses-PNC1/anat/sub-81231_ses-PNC1_T1w.nii.gz'

Can we replicate the same on Flywheel? Something to consider...

For now, a utility for refacing the T1 for each subject:

In [1]:
import flywheel
import subprocess
import os

In [2]:
client = flywheel.Client()

subject_label = 'sub-NDARBK082PDD'

project_label = 'ReproBrainChart'

proj = client.projects.find_first('label="ReproBrainChart"')

In [3]:
subjects = proj.subjects()

In [4]:
len(subjects)

5654

In [5]:
sub = client.lookup('bbl/{}/{}'.format(project_label, subject_label))

In [6]:
sessions = [client.get(x.id) for x in sub.sessions()]

In [7]:
sessions

[{'age': None,
  'analyses': [{'created': datetime.datetime(2020, 5, 24, 20, 49, 58, 924000, tzinfo=tzutc()),
                'description': None,
                'files': [],
                'gear_info': {'category': 'analysis',
                              'id': '5e62bf29830eb917ee084492',
                              'name': 'fw-heudiconv',
                              'version': '0.2.11_0.2.4'},
                'id': '5ecaddf61a0c1d1dbf169fb3',
                'info': None,
                'inputs': [],
                'job': {'attempt': 1,
                        'compute_provider_id': '5d8ed960507ac10001ff63e4',
                        'config': {'config': {'action': 'Validate',
                                              'default_heuristic': '',
                                              'do_whole_project': False,
                                              'dry_run': False,
                                              'extended_bids': True},
                         

In [107]:
def download_t1w(acquisition_obj, name, dest):
    '''
    download a file from an acquisition, using "dest" as the filename on disk
    '''
    
    acquisition_obj.download_file(name, dest)

In [108]:
def run_afni(path_to_file, filename, rec_label='refaced', dry_run=True, verbose=True):
    '''
    run afni reface, using rec_label as the value in the rec-<value> filename
    
    returns the returncode and std out
    '''
    
    assert os.path.isfile(path_to_file + filename)
    
    refaced_filename = filename.replace("_T1w.nii.gz", "_rec-{}_T1w.nii.gz".format(rec_label))
    process = ['@afni_refacer_run', '-input', path_to_file + filename ,'-mode_reface', '-prefix', path_to_file + refaced_filename]
    
    if dry_run:
        process.insert(0, 'echo')
        os.system('cp {} {}'.format(path_to_file + filename, path_to_file + refaced_filename))
    if verbose:
        print("Running:\n", " ".join(process))
    
    p = subprocess.Popen(process, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
    
    if out is None:
        out = ''
    return refaced_filename, p.returncode, out

In [114]:
def replace_t1w(acquisition_obj, original_info, replacement_filename, replacement_path, rec_label, delete=False):
    '''
    upload a refaced T1 to an acquisition to replace an original
    
    '''
    
    info = original_info.copy()
    
    info['BIDS']['Filename'] = replacement_filename
    info['BIDS']['Rec'] = rec_label
    
    acquisition_obj.upload_file(replacement_path + replacement_filename)
    
    acquisition_obj = acquisition_obj.reload()
    
    files = acquisition_obj.files
    
    target = [f for f in files if f.name == replacement_filename].pop()
    
    acquisition_obj.update_file_info(replacement_filename, info)
    
    if delete:
        print("deleting existing T1")
        acquisition_obj.delete_file(original_info['BIDS']['Filename'])
    
    return None

In [115]:
def process_subject(subject_obj, out_filename=None, destination='./', rec_label='refaced', verbose = True, dry_run=True):
    '''
    Process a subject:
       
       loop over sessions, acquisitions, files
       if a T1w is found, download it
       run afni reface and create a new file with 
    '''
    sessions = subject_obj.sessions()

    for ses in sessions:

        acquisitions = [client.get(a.id) for a in ses.acquisitions()]

        for acq in acquisitions:

            for f in acq.files:

                if get_nested(f, 'info', 'BIDS', 'Modality') == 'T1w':
                    
                    out_filename = f.name if out_filename is None else out_filename
                    
                    if verbose:
                        print("T1 found: {}\n\tDownloading as {} ...".format(f.name, out_filename))
                    
                    download_t1w(acq, f.name, destination + out_filename)
                    
                    if verbose:
                        print("Running afni reface...")

                    refaced_file, status, output = run_afni(destination, out_filename, rec_label, dry_run=dry_run, verbose=verbose)
                    
                    if status != 0:
                        print("There was a problem running afni")
                        if verbose:
                            print(output)
                    else:
                        if verbose:
                            print("Afni ran successfully!")
                            print("Uploading...")
                        replace_t1w(acq, f.info, refaced_file, destination, rec_label, delete=False)

In [117]:
process_subject(subjects[2], out_filename='foobar_T1w.nii.gz', destination="../data/", rec_label='refaced', verbose=True, dry_run=False)

T1 found: sub-NDARAC904DMU_ses-HBNsiteRU_acq-HCP_T1w.nii.gz
Running afni reface...
Running:
 @afni_refacer_run -input ../data/foobar_T1w.nii.gz -mode_reface -prefix ../data/foobar_rec-refaced_T1w.nii.gz
Afni ran successfully!


The above works by downloading from `fw`, running reface, and then uploading. We also need a plan for finding a subject's T1w in a BIDS dir on disk, running refacer, then uploading to `fw` and deleting existing.